In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
CUDA_LAUNCH_BLOCKING=1
raceDB = build_dataset('gru_inputs_new_simple.npy', hidden_size ,state_filter=states, margin_type='sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_']
(1199337, 22)
(126812, 24)
Latest date = 2023-04-05 00:00:00
num_features_per_dog=17


  0%|          | 0/3555 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/16794 [00:00<?, ?it/s]

c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
thorwing
number of races = 16794, number of unique dogs = 3555
0        (Auckland, 318.0)
1        (Auckland, 527.0)
2        (Auckland, 318.0)
3        (Auckland, 527.0)
4        (Auckland, 603.0)
               ...        
16789     (Waikato, 318.0)
16790     (Waikato, 527.0)
16791     (Waikato, 318.0)
16792     (Waikato, 527.0)
16793     (Waikato, 318.0)
Length: 16794, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 13565, Test examples 3229


In [8]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [9]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [5094, 5162, 3295]


In [10]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in dog.races.keys() if x in batch]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

0
['500543147', '500543148', '500543150', '500543149', '500543144', '500543145', '500543146', '500543151', '500543152', '500543143', '500543141', '500543140', '500543142', '500539136', '500539137', '500539139', '500539134', '500539135', '500539133', '500539138', '500539141', '500539142', '500539143', '500539140', '500539132', '500539131', '500539144', '500539130', '500539129', '500541175', '500541174', '500541170', '500541168', '500541176', '500541177', '500541172', '500541171', '500541173', '500541178', '500541169', '500541179', '500541180', '500541182', '500541181', '501809115', '501809126', '501809127', '501809117', '501809125', '501809124', '501809129', '501809128', '501809118', '501809120', '501809123', '501809116', '501809121', '501809122', '501809119', '502577690', '502577687', '502577686', '502577685', '502577689', '502577688', '502563668', '502563667', '502563669', '502577695', '502563670', '502577694', '502577692', '502577691', '502563678', '502563672', '502563677', '50257769

In [11]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [12]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [1947, 1877, 1553]
Train examples [1947, 1877, 1553]
Train examples [5094, 5162, 3295]
Train examples [5094, 5162, 3295]


In [13]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/3555 [00:00<?, ?it/s]

In [14]:
raceDB.attach_races_to_dog_inputv2()

  0%|          | 0/3555 [00:00<?, ?it/s]

In [15]:
# batchnorm = torch.nn.BatchNorm1d(84).to('cuda:0')

In [16]:
def batch_norm_pack_seq(seq):
    x,idx = pad_packed_sequence(seq)
    print(x.shape)
    x = x.transpose(0,1).transpose(1,2)
    x = batchnorm(x.float())
    x = x.transpose(2,1).transpose(1,0)
    x = unpad_sequence(x, idx)
    x = pack_sequence(x, enforce_sorted=False)
    return x

In [17]:
# test_idx = range(0,len(raceDB.test_dog_ids))
# # packed_x = [batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False)).to('cuda:0') for x in train_dog_input]
# packed_y = batch_norm_pack_seq(pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False))

In [18]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = [pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [19]:
train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [20]:
[sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

[20.19774011299435, 20.730420884389986, 15.928525434642626]

In [21]:
unpack_sequence(packed_x[0])[0][2]

tensor([318.0000,   0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          6.0000,   0.0000,   1.0000,   0.0000,  32.7000,   1.0000,  17.0999,
         -1.0000,  -1.0000,   8.9000,   0.0000,  -2.0000,   2.0000,   0.0000,
         33.3000,  21.9300,  -1.0000], device='cuda:0', dtype=to

In [22]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids,'packed_x':packed_x, 'packed_y':packed_y}

In [23]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = rnn_classes.GRUNetv2(87,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/silver-surf-359_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      # optimizer = optim.AdamW(model.parameters(), lr=0.0001, set_t)
      # optimizer = optim.Adadelta(model.parameters())
      optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt, Exception) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru_v6 , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [24]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'stats':raceDB.stats_cols, 'num_layers':2,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0008, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [25]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [26]:
raceDB.reset_hidden()

In [27]:
for race in raceDB.racesDict.values():
    race.hidden_in = 0

In [28]:
torch.cuda.empty_cache()

In [29]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


{'hidden_size': 32, 'stats': ['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0008, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'hidden_size': 32, 'stats': ['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_'], 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0008, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
GRUNetv2(
  (gru): GRU(87, 32, num_layers=2)
  (batchnorm): BatchNorm1d(326, eps=1e-05, mo

  0%|          | 0/10000 [00:00<?, ?it/s]

created path
The wandb backend process has shutdown
finished Early


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x000001E61E5A33A0> (for post_run_cell):


Exception: The wandb backend process has shutdown

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\wandb_run.py", line 170, in check_status
    status_response = self._interface.communicate_stop_status()
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface.py", line 114, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 394, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 225, in _communicate
    return self._communicate_asyn

In [43]:
races = raceDB.batches['batch_races'][2]

In [55]:
1/max(torch.tensor(-1),torch.tensor(0.01))

tensor(100.)

In [52]:
(1-1/(max(torch.tensor(-1),torch.tensor(0.01))))

tensor(-99.)

In [50]:
[r.win_price_weight  for r in races][3].item()

-1.0

In [51]:
[r.win_price_weightv2  for r in races][3].item()

-99.0

In [44]:
[r.prices for r in races]

[[4.504666281656208,
  5.167040450421773,
  200.0,
  2.5078112921528737,
  16.514119836411354,
  27.131130903,
  44.832309584607714,
  12.2220010878111],
 [17.766809040596705,
  24.0,
  139.24579684750177,
  2.9268698833702365,
  3.4,
  300.0,
  25.14685064414829,
  4.8],
 [37.92115936262999,
  7.2,
  2.967198063857504,
  27.074693750490983,
  5.332126513138647,
  8.8,
  36.0,
  8.2],
 [-1.0, -1.0, -1.0, -1.0, -1.0, 0, -1.0, -1.0],
 [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0],
 [3.130361066792465,
  21.0,
  17.02670858,
  3.104578477674732,
  55.0,
  16.462553294377617,
  23.424683563198435,
  6.812231557430932],
 [8.8,
  6.678489245,
  27.280542056613893,
  0,
  42.640418996,
  2.68,
  4.455419657620369,
  15.799185208],
 [4.6,
  29.31412230812013,
  21.0,
  121.99636399026956,
  3.330662446,
  22.110832185548052,
  3.289732137198734,
  27.725538670527012],
 [4.631058443524523,
  9.482388404660762,
  6.8,
  3.0771071748080425,
  87.089328799,
  16.5,
  16.0,
  12.0],
 [14.9041575

In [29]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [250]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.002,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [7]
        },
        "batch_before_backwards": {
            'values': [7]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_before_backwards': {'values': [7]},
                'batch_size': {'values': [7]},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [250]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'hidden_size': {'value': 32},
                'l1_beta': {'value': 0.1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.002,
                                  'min': 1e-05},
                'len_data': {'value': 16794},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'len_data': {'value': 16794},
  'hidden_size': {'value': 32},
  'epochs': {'values': [250]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['CEL']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 1e-05, 'max': 0.002},
  'l1_beta': {'value': 0.1},
  'batch_size': {'values': [7]},
  'batch_before_backwards': {'values': [7]}}}

In [30]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ma3z2pck
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/ma3z2pck


wandb: Agent Starting Run: d5q5n509 with config:
wandb: 	batch_before_backwards: 7
wandb: 	batch_size: 7
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 32
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0010677522892121151
wandb: 	len_data: 16794
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0010677522892121151, 'len_data': 16794, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0010677522892121151, 'len_data': 16794, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNetv2(
  (gru): GRU(87, 32, num_layers=2)
  (batchnorm): BatchNorm1d(326, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=32, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=326, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear

  0%|          | 0/250 [00:00<?, ?it/s]

...

created path



ROI < 30,█▁▁▂▆▄▃▄▃▆▅▆▇
ROI < 30 2,▇▁▁▁▆▅▄▅▄▇▅▇█
accuracy,▁▃▅▅▇▇███████
loss_1,█▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▂▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁
loss_val,█▂▂▂▁▁▁▁▁▁▁▁▁
multibet outlay < 30,█▂▁▂▁▁▁▁▁▁▁▁▁
multibet profit,▂▁▂▁▅▇█▆▆▇▇▇█
multibet profit < 30,█▁▁▁▆▄▃▄▃▆▅▆▇
multibet profit < 30 sd,█▁▁▁▁▁▁▁▁▁▁▁▁
ROI < 30,-0.00321
ROI < 30 2,0.03014


wandb: Agent Starting Run: bggb0ff4 with config:
wandb: 	batch_before_backwards: 7
wandb: 	batch_size: 7
wandb: 	dropout: 0.3
wandb: 	epochs: 250
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	hidden_size: 32
wandb: 	l1_beta: 0.1
wandb: 	learning_rate: 0.0004724711894789964
wandb: 	len_data: 16794
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0004724711894789964, 'len_data': 16794, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
250
{'batch_before_backwards': 7, 'batch_size': 7, 'dropout': 0.3, 'epochs': 250, 'f1_layer_size': 256, 'f2_layer_size': 64, 'hidden_size': 32, 'l1_beta': 0.1, 'learning_rate': 0.0004724711894789964, 'len_data': 16794, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
GRUNetv2(
  (gru): GRU(87, 32, num_layers=2)
  (batchnorm): BatchNorm1d(326, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=32, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=326, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear

  0%|          | 0/250 [00:00<?, ?it/s]